# Computational Theory Problems

In [1]:
#Library to work with arrays and mathematical functions
import numpy as np


## Probelm 1 Binary Words and Operations

This section implements the 32-bit word primitives used by SHA-256. All operations conceptually act on unsigned 32-bit words. In Python, integers are unbounded, so either mask with `0xFFFFFFFF` to keep values in 32 bits, or use [`numpy.uint32`](https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32) for natural 32-bit wraparound.

Operations and definitions:
- Logical right shift: SHR^n(x) — shift right by n bits, zero-fill on the left.
- Rotate right: ROTR^n(x) — shift right by n bits, wrapping the bits that fall off back to the left.
- Choose: Ch(x, y, z) = (x AND y) XOR ((NOT x) AND z) — per bit, pick y when x’s bit is 1, else z.
- Majority: Maj(x, y, z) = (x AND y) XOR (x AND z) XOR (y AND z) — per bit, returns the majority among x, y, z.
- Big sigmas:
  - Σ0(x) = ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x)
  - Σ1(x) = ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x)
- Small sigmas:
  - σ0(x) = ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x)
  - σ1(x) = ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x)


In [2]:
def ROTR(x, n):
    """
    Rotate right (Circular right shift)
    32-bit integer x rotated right by n bits.
    """
    return ((x >> n) | (x << (32 - n)))

In [3]:
def Parity(x,y,z):
    """
    Parity returns the bitwise parity (XOR) of three integers.
    """
    return x ^ y ^ z

In [4]:
def Ch(x,y,z):
    """
    SHA-256 'choose' function.
    For each bit, returns y if the corresponding bit of x is 1, else z.
    """
    return (x & y) ^ (~x & z)

In [5]:
def Maj(x,y,z):
    """
    SHA-256 'majority' function.
    For each bit, returns the majority value among x, y, and z.
    """
    return (x & y) ^ (x & z) ^(y & z)

In [6]:
def Sigma0(x):
    """
    SHA256 Sigma0 function
    For a 32-bit integer x, performs right rotations by 2, 13, and 22 bits
    and returns the XOR of the results.
    """
    return (ROTR(x, 2)) ^ (ROTR(x, 13)) ^ (ROTR(x, 22))

In [7]:
def Sigma1(x):
    """
    SHA256 Sigma1 function
    For a 32-bit integer x, performs right rotations by 6, 11 and 25 bits
    and returns the XOR of the results.
    """

    return (ROTR(x, 6)) ^ (ROTR(x, 11)) ^ (ROTR(x, 25))

In [8]:
def sigma0(x):
    """
    SHA-256 'sigma0' function.
    For a 32-bit integer x, performs right rotations by 7, 18, and shift 3 bits
    and returns the XOR of the results.
    """
    return (ROTR(x, 7)) ^ (ROTR(x, 18)) ^ (x >> 3)

In [9]:
def sigma1(x):
    """
    SHA-256 'sigma1' function.
    For a 32-bit integer x, performs right rotations by 17, 19, and shift 10 bits
    and returns the XOR of the results.
    """
    return (ROTR(x, 17)) ^ (ROTR(x, 19)) ^ (x >> 10)

## Problem 2 Fractional Parts of Cube Roots

Goal: Derive 64 constant 32‑bit words from the fractional parts of the cube roots of the first 64 primes. In SHA‑256 these become the constants K[0..63] (FIPS 180‑4, Section 4.2.2).

Process implemented:
1. Generate the first 64 primes starting at 2.
2. For each prime p compute its real cube root c = p ** (1/3).
3. Separate fractional part: frac = c - int(c).
4. Scale to 32 bits: frac32 = floor(frac * 2**32). This captures the leading 32 bits of the fractional part.

Why multiply by 2**32:
- The spec defines each word as the first 32 bits of the fractional part, multiplying by 2^32 and truncating shifts those bits into the integer portion.


Reference:
- FIPS PUB 180‑4 Secure Hash Standard: https://csrc.nist.gov/publications/detail/fips/180/4/final

In [10]:
#https://stackoverflow.com/questions/54543956/finding-prime-number-using-the-square-root-method
#code adapted from above link
def is_prime(num):
    """
    Check if a number is prime

    This functions checks if a given number is prime,
    it divides the number by integers between 2 and 1 more than its square root
    as if its not divisible by a factor smaller than the square root than it cant be divisible by a larger factor than the square root
    if it passes this test then it is prime
    """
    if num >= 2:
        for i in range(2, int(np.sqrt(num)) + 1):
            if (num % i) == 0:
                return False
    return True


In [11]:
def primes(n):
    """
    Generate the first n prime numbers
    start from 2 and check each number if it is prime using the is_prime function
    if it is prime add it to the list of primes
    add 1 to the number and repeat until we have n primes
    """
    
    primes = []
    num = 2  
    while len(primes) < n:
        if is_prime(num):
            primes.append(num)
        num += 1

    return primes

In [12]:
def frac32():
    """
    Calculate the 32-bit fractional parts of the cube roots of the first 64 prime numbers
    1. Get the first 64 prime numbers using the primes function
    2. For each prime number, calculate its cube root
    3. Isolate the fractional part of the cube root by subtracting the integer part
    4. Convert the fractional part to a 32-bit integer by multiplying by 2^32 and taking the integer part
    5. Store the 32-bit integers in a list and return the list
    """
    #Get the first 64 prime numbers
    prime_numbers = primes(64)
    #List to hold the 32-bit fractional parts of the cube roots
    frac32 = []

    for prime in prime_numbers:
        #Calculate the cube root of the prime number
        cube_root = prime ** (1/3)
        #Int fun is used here to isolate the fractional part from the cube root by removing the integer part
        frac = cube_root - int(cube_root)
        #Convert the fractional part to a 32-bit integer and removing the left over fractional part
        frac32.append(int(frac * (2**32)))
        
    return frac32

In [13]:
for frac in frac32():
    print(f"{frac:08x}")

428a2f98
71374491
b5c0fbcf
e9b5dba5
3956c25b
59f111f1
923f82a4
ab1c5ed5
d807aa98
12835b01
243185be
550c7dc3
72be5d74
80deb1fe
9bdc06a7
c19bf174
e49b69c1
efbe4786
0fc19dc6
240ca1cc
2de92c6f
4a7484aa
5cb0a9dc
76f988da
983e5152
a831c66d
b00327c8
bf597fc7
c6e00bf3
d5a79147
06ca6351
14292967
27b70a85
2e1b2138
4d2c6dfc
53380d13
650a7354
766a0abb
81c2c92e
92722c85
a2bfe8a1
a81a664b
c24b8b70
c76c51a3
d192e819
d6990624
f40e3585
106aa070
19a4c116
1e376c08
2748774c
34b0bcb5
391c0cb3
4ed8aa4a
5b9cca4f
682e6ff3
748f82ee
78a5636f
84c87814
8cc70208
90befffa
a4506ceb
bef9a3f7
c67178f2


## Problem 3 Padding

## Problem 4 Hashes 

## Problem 5 Passwords

## End